In [1]:
date = 20211231  #curent date

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

RPSIA = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\10.0 RPSIA\\'+str(date)[:4]+'\\RPSIA_'+str(date)[:6]+'.xlsx', index = False)
Total_Impaired = pd.read_excel (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\"+str(date)[:4]+"\\"+str(date)[:6]+"\working\(MIB) Total_impaired_"+str(date)[:6]+".xlsx")
Total_Impaired_P = pd.read_excel (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\"+str(date)[:4]+"\\"+str(date-100)[:6]+"\working\(MIB) Total_impaired_"+str(date-100)[:6]+".xlsx")#, sep = ",", header = 0)

cust_iscparent = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\99_Adhoc\Syahid\A\User defined\cust_iscparent.xlsx", index = False)

location_cust = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\02_Teradata\01_Credit_Risk_Reporting\01_Cust\\"+str(date)+"\\S001_004_ACRM_CUST"+str(date)+".txt"
cust = pd.read_csv(location_cust, sep = "|", header = 0, error_bad_lines = False)
cust.columns = cust.columns.str.replace(" ", "_")
cust.M_SUB_SUB_MARKET_SEGMENT.replace({'    ': -9999}, inplace=True)

Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)
Total_Impaired_P.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired_P.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)
RPSIA.V_ORIGINAL_ACCOUNT_NUMBER = RPSIA.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)

Total_Impaired.CUSTOMER_ID = Total_Impaired.CUSTOMER_ID.astype(str)
Total_Impaired_P.CUSTOMER_ID = Total_Impaired_P.CUSTOMER_ID.astype(str)

Total_Impaired_P.CUSTOMER_ID = Total_Impaired_P.CUSTOMER_ID.str.strip()
Total_Impaired.CUSTOMER_ID = Total_Impaired.CUSTOMER_ID.str.upper()

Total_Impaired['OUTSTANDING_AFTER_EIR'] = Total_Impaired.OUTSTANDING + Total_Impaired.EIR
Total_Impaired['OUTSTANDING_AFTER_EIR_UWI'] = Total_Impaired.OUTSTANDING\
+ Total_Impaired.EIR + Total_Impaired.MOD_GAIN_LOSS - Total_Impaired.UWI_CLOSING_BAL

RPSIA['F_RPSIA'] = 'Y'

Total_Impaired_P['F_NCIF_CUST'] = 'N'

Total_Impaired_P['F_NCIF_ACCT'] = 'N'


L01 = Total_Impaired.merge(RPSIA[['V_ORIGINAL_ACCOUNT_NUMBER','F_RPSIA']], on = 'V_ORIGINAL_ACCOUNT_NUMBER', how = 'left')
L01_1 = L01.merge(Total_Impaired_P[['CUSTOMER_ID','F_NCIF_CUST']], on = 'CUSTOMER_ID', how = 'left')
L01_2 = L01_1.merge(Total_Impaired_P[['V_ORIGINAL_ACCOUNT_NUMBER','F_NCIF_ACCT']], on = 'V_ORIGINAL_ACCOUNT_NUMBER', how = 'left')
L01_2 = L01_2.fillna(0)

L01_2.loc[L01_2.F_RPSIA == 0, 'F_RPSIA'] = 'N'
L01_2.loc[L01_2.F_NCIF_CUST == 0, 'F_NCIF_CUST'] = 'Y'
L01_2.loc[L01_2.F_NCIF_ACCT == 0, 'F_NCIF_ACCT'] = 'Y'

L1 = L01_2.drop_duplicates()

def NOB_GROUP(V_D_CUST_INDUSTRY,MKT_SUB_SEGMENT):
    if V_D_CUST_INDUSTRY=='9999':
        return '9999'
    elif V_D_CUST_INDUSTRY[0] == '1': 
        return '1000'
    elif V_D_CUST_INDUSTRY[0] == '2': 
        return '2000'
    elif V_D_CUST_INDUSTRY[0] == '3': 
        return '3000'
    elif V_D_CUST_INDUSTRY[0] == '4': 
        return '4000'
    elif V_D_CUST_INDUSTRY[0] == '5': 
        return '5000'
    elif V_D_CUST_INDUSTRY[0] == '6': 
        return '6000'
    elif V_D_CUST_INDUSTRY[0] == '7': 
        return '7000'
    elif V_D_CUST_INDUSTRY[0] == '8': 
        return '8000'
    elif V_D_CUST_INDUSTRY[0] == '9': 
        return '9000'
    elif MKT_SUB_SEGMENT == '9':
        return "9000" # Added 18/9/2018 
    else: 
        return '0000'
L1['NOB_GROUP'] = L1.apply(lambda x: NOB_GROUP(x['V_D_CUST_INDUSTRY'], x['MKT_SUB_SEGMENT']), axis=1)

def NOB_GROUP_DESC(V_D_CUST_INDUSTRY,MKT_SUB_SEGMENT):
    if V_D_CUST_INDUSTRY=='9999':
        return 'SECTOR N.E.C'
    elif V_D_CUST_INDUSTRY[0] == '1': 
        return 'PRIMARY AGRICULTURE'
    elif V_D_CUST_INDUSTRY[0] == '2': 
        return 'MINING AND QUARRYING'
    elif V_D_CUST_INDUSTRY[0] == '3': 
        return 'MANUFACTURING (INCLUDING AGRO-BASED)'
    elif V_D_CUST_INDUSTRY[0] == '4': 
        return 'ELECTRICITY, GAS AND WATER SUPPLY'
    elif V_D_CUST_INDUSTRY[0] == '5': 
        return 'CONSTRUCTION'
    elif V_D_CUST_INDUSTRY[0] == '6': 
        return 'WHOLESALE AND RETAIL TRADE, AND RESTAURANT AND HOTELS'
    elif V_D_CUST_INDUSTRY[0] == '7': 
        return 'TRANSPORT, STORAGE AND COMMUNICATION'
    elif V_D_CUST_INDUSTRY[0] == '8': 
        return 'FINANCE, REAL ESTATE AND BUSINESS ACTIVITIES'
    elif V_D_CUST_INDUSTRY[0] == '9': 
        return 'EDUCATION, HEALTH AND OTHERS'
    elif MKT_SUB_SEGMENT == '9':
        return "EDUCATION, HEALTH AND OTHERS" # Added 18/9/2018 
    else: 
        return 'HOUSEHOLD'
L1['NOB_GROUP_DESC'] = L1.apply(lambda x: NOB_GROUP_DESC(x['V_D_CUST_INDUSTRY'], x['MKT_SUB_SEGMENT']), axis=1)

def PRODUCT_GROUP(PROD_LV5,PROD_LV6,MKT_SUB_SEGMENT,V_PROD_CODE):
    if ((PROD_LV5 == 'Floor Stocking') | (PROD_LV5 == 'Hire Purchase')):
        return '09.HP'
    elif (PROD_LV5 == 'Credit Cards' ):
        return '10.CC'; ELSE
    elif ((MKT_SUB_SEGMENT == 'CORPORATE BANKING') & (PROD_LV6 == 'Rent-To-Own')):
        return '03.CB-RTO'; # /*JUST ADDED 9 DEC 2019*/
    elif MKT_SUB_SEGMENT == 'CORPORATE BANKING' :
        return '01.CB' 
    elif PROD_LV5 == 'SMF':
        return '02.CB-SMF'
    elif MKT_SUB_SEGMENT == 'BUSINESS BANKING': 
        return '04.BB'
    elif MKT_SUB_SEGMENT == 'SME BANKING':
        return '05.SME'
    elif ((MKT_SUB_SEGMENT == 'CONSUMER') & (PROD_LV6 == 'Housing' )):
        return '06.HOME'
    elif ((MKT_SUB_SEGMENT == 'CONSUMER') & (PROD_LV6 == 'Shophouse')): 
        return '07.SHOPHOUSE'
    elif ((MKT_SUB_SEGMENT == 'CONSUMER') & (PROD_LV6 == 'Other Mortgage Loan')): 
        return '08.OTHMORTGAGE'
    elif ((MKT_SUB_SEGMENT == 'CONSUMER') & ((V_PROD_CODE == '711')|(V_PROD_CODE == '714'))):
        return '11.CC-PF'
    elif( (MKT_SUB_SEGMENT == 'CONSUMER') & (PROD_LV5 == 'Unit Trust' )):
        return '12.UT'
    elif ((MKT_SUB_SEGMENT == 'CONSUMER') & (PROD_LV6 == 'Salary')) :
        return '13.SALARY'
    elif MKT_SUB_SEGMENT == 'CONSUMER' :
        return '14.OTHRETAIL'
    else:
        return '99.CHECK'
L1['PRODUCT_GROUP'] = L1.apply(lambda x: PRODUCT_GROUP(x['PROD_LV5'], x['PROD_LV6'], x['MKT_SUB_SEGMENT'],\
                                                         x['V_PROD_CODE']), axis=1)
L1.columns = L1.columns.str.replace(":", "_")

cust.columns = cust.columns.str.replace(" ", "_")
cust = cust.fillna(0)
cust.M_SUB_SUB_MARKET_SEGMENT.replace({'    ': 0}, inplace=True) 

cust_iscparent = cust_iscparent.fillna(0)

def NOB_GROUP2(NOB_GROUP,NOB_GROUP_DESC,MKT_SUB_SEGMENT):
    if (NOB_GROUP =="0000")&(NOB_GROUP_DESC=="HOUSEHOLD")&(MKT_SUB_SEGMENT!="CONSUMER"):
        return "9999"
    else:
        return NOB_GROUP
L1['NOB_GROUP2'] = L1.apply(lambda x: NOB_GROUP2(x['NOB_GROUP'], x['NOB_GROUP_DESC'], x['MKT_SUB_SEGMENT']), axis=1)

def NOB_GROUP_DESC2(NOB_GROUP,NOB_GROUP_DESC,MKT_SUB_SEGMENT):
    if (NOB_GROUP =="0000")&(NOB_GROUP_DESC=="HOUSEHOLD")&(MKT_SUB_SEGMENT!="CONSUMER"):
        return "SECTOR N.E.C"
    else:
        return NOB_GROUP_DESC
L1['NOB_GROUP_DESC2'] = L1.apply(lambda x: NOB_GROUP_DESC2(x['NOB_GROUP'], x['NOB_GROUP_DESC'], x['MKT_SUB_SEGMENT']), axis=1)

def NPF_TAG(F_EXPOSURE_DEFAULT_STATUS_FLAG):
    if F_EXPOSURE_DEFAULT_STATUS_FLAG == 'Y':
        return 'NPF'
    elif F_EXPOSURE_DEFAULT_STATUS_FLAG == 'N':
        return 'PF'
    else:
        return 'NA'
L1['NPF_TAG'] = L1.F_EXPOSURE_DEFAULT_STATUS_FLAG.apply(NPF_TAG)

cust1 = cust[['CUS_NO','INDUSTRIAL_SECTOR']]
cust1.CUS_NO = cust1.CUS_NO.astype(str)
L01_3 = L1.merge(cust1.rename(columns={'CUS_NO': 'CUSTOMER_ID'}), on = 'CUSTOMER_ID', how = 'left')
L01_3 = L01_3.fillna(0)

L01_3.columns = L01_3.columns.str.replace("(", "_").str.replace(")", "_").str.replace(".", "_").str.replace("-", "_").str.replace("\n", "_").str.replace(" ", "_")

cust_iscparent = cust_iscparent.drop_duplicates('ISC_CODE', keep='last')


L01_3.INDUSTRIAL_SECTOR = L01_3.INDUSTRIAL_SECTOR.str.strip()

Total_Impaired_Final = pd.merge(L01_3,cust_iscparent[['ISC_CODE','ISC_PARENT']].rename(columns={'ISC_CODE': 'INDUSTRIAL_SECTOR'}),\
                    on = 'INDUSTRIAL_SECTOR', how = 'left')
Total_Impaired_Final = Total_Impaired_Final.fillna(0)

Total_Impaired_Final.loc[Total_Impaired_Final.ISC_PARENT == 0, 'ISC_PARENT'] = '23. Not Populated'

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [2]:
SUMM_IF = Total_Impaired_Final.rename(columns={'V_D_CUST_INDUSTRY':'NATURE_OF_BUSINESS'})
SUMM_IF['NOA'] = 1

SUMM_IF = SUMM_IF.fillna(0).groupby(['NATURE_OF_BUSINESS'\
,'NOB_GROUP2'\
,'NOB_GROUP_DESC2'\
,'NOB_GROUP'\
,'NOB_GROUP_DESC'\
,'MKT_SUB_SEGMENT'\
,'PROD_LV1'\
,'PROD_LV5'\
,'PROD_LV6'\
,'PRODUCT_GROUP'\
,'F_RPSIA'\
,'F_NCIF_CUST'\
,'F_NCIF_ACCT'])[['NOA'\
,'UWI_CLOSING_BAL'\
,'OUTSTANDING'\
,'OUTSTANDING_AFTER_EIR'\
,'OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()


In [3]:
LIST_IF_PDT = Total_Impaired_Final.iloc[np.where((Total_Impaired_Final.PRODUCT_GROUP.isin(['01.CB','04.BB','05.SME']))&(Total_Impaired_Final.PROD_LV1=='On Balance Sheet'))]
LIST_IF_PDT['IF_FLAG'] = 'Y'

LIST_IF_PDT = LIST_IF_PDT[['V_ORIGINAL_ACCOUNT_NUMBER'\
,'CUSTOMER_ID'\
,'V_D_CUST_FIRST_NAME'\
,'V_D_CUST_INDUSTRY'\
,'INDUSTRIAL_SECTOR'\
,'ISC_PARENT'\
,'PRODUCT_GROUP'\
,'F_RPSIA'\
,'F_NCIF_CUST'\
,'OUTSTANDING'\
,'OUTSTANDING_AFTER_EIR'\
,'OUTSTANDING_AFTER_EIR_UWI'\
,'IF_FLAG']].sort_values(by=['PRODUCT_GROUP','OUTSTANDING_AFTER_EIR_UWI'],ascending=[True,True])

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
LIST_IF_BYCUS = Total_Impaired_Final.iloc[np.where((Total_Impaired_Final.PRODUCT_GROUP.isin(['01.CB','04.BB','05.SME']))&(Total_Impaired_Final.PROD_LV1=='On Balance Sheet'))]
LIST_IF_BYCUS['IF_FLAG'] = 'Y'

LIST_IF_BYCUS = LIST_IF_BYCUS.fillna(0).groupby(['CUSTOMER_ID'\
,'V_D_CUST_FIRST_NAME'\
,'V_D_CUST_INDUSTRY'\
,'INDUSTRIAL_SECTOR'\
,'ISC_PARENT'\
,'PRODUCT_GROUP'\
,'F_RPSIA'\
,'F_NCIF_CUST','IF_FLAG'])[['OUTSTANDING'\
,'OUTSTANDING_AFTER_EIR'\
,'OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

LIST_IF_BYCUS = LIST_IF_BYCUS[['CUSTOMER_ID'\
,'V_D_CUST_FIRST_NAME'\
,'V_D_CUST_INDUSTRY'\
,'INDUSTRIAL_SECTOR'\
,'ISC_PARENT'\
,'PRODUCT_GROUP'\
,'F_RPSIA'\
,'F_NCIF_CUST'\
,'OUTSTANDING'\
,'OUTSTANDING_AFTER_EIR'\
,'OUTSTANDING_AFTER_EIR_UWI'\
,'IF_FLAG']]

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
SUMM_IF_PDT = Total_Impaired_Final.iloc[np.where(Total_Impaired_Final.PROD_LV1=='On Balance Sheet')]

SUMM_IF_PDT = SUMM_IF_PDT.fillna(0).groupby(['PRODUCT_GROUP'])[['OUTSTANDING'
,'OUTSTANDING_AFTER_EIR'
,'OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

In [6]:
SUMM_IF_NOB = Total_Impaired_Final.iloc[np.where(Total_Impaired_Final.PROD_LV1=='On Balance Sheet')]

SUMM_IF_NOB = SUMM_IF_NOB.fillna(0).groupby(['NOB_GROUP2'\
,'NOB_GROUP_DESC2'])[['OUTSTANDING'\
,'OUTSTANDING_AFTER_EIR'\
,'OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

In [7]:
SUMM_IF_ISC = Total_Impaired_Final.iloc[np.where(Total_Impaired_Final.PROD_LV1=='On Balance Sheet')]

SUMM_IF_ISC = SUMM_IF_ISC.fillna(0).groupby(['ISC_PARENT'])[['OUTSTANDING'\
,'OUTSTANDING_AFTER_EIR'\
,'OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

In [8]:
SUMM_IF_HOUSEHOLD = Total_Impaired_Final.iloc[np.where((Total_Impaired_Final.NOB_GROUP2=='0000')&(Total_Impaired_Final.PROD_LV1=='On Balance Sheet'))]

SUMM_IF_HOUSEHOLD = SUMM_IF_HOUSEHOLD.fillna(0).groupby(['PRODUCT_GROUP'])[['OUTSTANDING'\
,'OUTSTANDING_AFTER_EIR'\
,'OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

In [9]:
SUMM_NCIF_PDT = Total_Impaired_Final.iloc[np.where((Total_Impaired_Final.F_NCIF_CUST=='Y')&(Total_Impaired_Final.PROD_LV1=='On Balance Sheet'))] 

SUMM_NCIF_PDT = SUMM_NCIF_PDT.fillna(0).groupby(['PRODUCT_GROUP'])[['OUTSTANDING'\
,'OUTSTANDING_AFTER_EIR'\
,'OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

In [10]:
LIST_NCIF_PDT = Total_Impaired_Final.iloc[np.where((Total_Impaired_Final.F_NCIF_CUST=='Y')&(Total_Impaired_Final.PRODUCT_GROUP.isin(['01.CB','04.BB','05.SME']))&(Total_Impaired_Final.PROD_LV1=='On Balance Sheet'))]

LIST_IF_PDT = LIST_IF_PDT[['V_ORIGINAL_ACCOUNT_NUMBER'\
,'CUSTOMER_ID'\
,'V_D_CUST_FIRST_NAME'\
,'V_D_CUST_INDUSTRY'\
,'INDUSTRIAL_SECTOR'\
,'ISC_PARENT'\
,'PRODUCT_GROUP'\
,'F_RPSIA'\
,'OUTSTANDING' \
,'OUTSTANDING_AFTER_EIR'\
,'OUTSTANDING_AFTER_EIR_UWI']].sort_values(by=['PRODUCT_GROUP','OUTSTANDING_AFTER_EIR_UWI'],ascending=[True,True])

In [11]:
Total_Impaired_Final.to_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(date)[:4]+'\\'+str(date)[:6]+'\working\Total_Impaired_Final_'+str(date)+'.txt', index = False)

writer = pd.ExcelWriter(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\"+str(date)[:4]+"\\"+str(date)[:6]+"\working\SUMM_IF_"+str(date)[:6]+"(python).xlsx", engine='xlsxwriter')


SUMM_IF.to_excel(writer, sheet_name='SUMM', index = False)

LIST_IF_PDT.to_excel(writer, sheet_name='LIST_IF_PDT', index = False)

LIST_IF_BYCUS.to_excel(writer, sheet_name='LIST_IF_BYCUS', index = False)

SUMM_IF_PDT.to_excel(writer, sheet_name='PDT', index = False)

SUMM_IF_NOB.to_excel(writer, sheet_name='NOB', index = False)

SUMM_IF_ISC.to_excel(writer, sheet_name='ISC', index = False)

SUMM_IF_HOUSEHOLD.to_excel(writer, sheet_name='HOUSEHOLD', index = False)

SUMM_NCIF_PDT.to_excel(writer, sheet_name='NCIF_PDT', index = False)

LIST_NCIF_PDT.to_excel(writer, sheet_name='LIST_NCIF_PDT', index = False)


writer.save()


In [12]:
#=======================================================RPSIA

donn = sqlite3.connect(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\"+str(date)[:4]+"\\"+str(date)[:6]+"\IF_"+str(date)+".db")
d = donn.cursor()

RPSIA.columns = RPSIA.columns.str.replace("(", "_").str.replace(")", "_").str.replace(".", "_")


RPSIA1 = RPSIA[['V_ORIGINAL_ACCOUNT_NUMBER','CUSTOMER_ID','V_D_CUST_FIRST_NAME','V_D_CUST_INDUSTRY','RPSIA_THIRD_PARTY','OUTSTANDING','GROSS_LOAN','EIR','MOD_GAIN_LOSS','UWI_CLOSING_BAL']]
RPSIA1.CUSTOMER_ID = RPSIA1.CUSTOMER_ID.astype(str)

cust1 = cust[['CUS_NO','INDUSTRIAL_SECTOR']]
cust1.CUS_NO = cust1.CUS_NO.astype(str)

RPSIA_ACCTLEVEL = RPSIA1.merge(cust1.rename(columns={'CUS_NO': 'CUSTOMER_ID'}), on = 'CUSTOMER_ID', how = 'left')
RPSIA_ACCTLEVEL['OUTSTANDING_AFTER_EIR'] = RPSIA_ACCTLEVEL.OUTSTANDING + RPSIA_ACCTLEVEL.EIR
RPSIA_ACCTLEVEL['OUTSTANDING_AFTER_EIR_UWI'] = RPSIA_ACCTLEVEL.OUTSTANDING + RPSIA_ACCTLEVEL.EIR + RPSIA_ACCTLEVEL.MOD_GAIN_LOSS - RPSIA_ACCTLEVEL.UWI_CLOSING_BAL

RPSIA_ACCTLEVEL1 = RPSIA_ACCTLEVEL.drop(columns=['EIR', 'MOD_GAIN_LOSS', 'UWI_CLOSING_BAL'])

RPSIA_ACCTLEVEL1 = RPSIA_ACCTLEVEL1[['V_ORIGINAL_ACCOUNT_NUMBER','CUSTOMER_ID','V_D_CUST_FIRST_NAME', 'V_D_CUST_INDUSTRY','RPSIA_THIRD_PARTY','INDUSTRIAL_SECTOR','OUTSTANDING','GROSS_LOAN','OUTSTANDING_AFTER_EIR','OUTSTANDING_AFTER_EIR_UWI']]

d.execute("CREATE TABLE IF NOT EXISTS RPSIA_ACCTLEVEL ({})".format(','.join(RPSIA_ACCTLEVEL1.columns)))

for row in RPSIA_ACCTLEVEL1.iterrows():
    sql = "INSERT INTO RPSIA_ACCTLEVEL({}) VALUES ({})".format(','.join(RPSIA_ACCTLEVEL1.columns), ','.join(['?']*len(RPSIA_ACCTLEVEL1.columns)))
    d.execute(sql, tuple(row[1]))

donn.commit()

d.execute("""
CREATE TABLE IF_RPSIA_ACCTLEVEL2 AS SELECT
DISTINCT V_ORIGINAL_ACCOUNT_NUMBER
,CUSTOMER_ID
,V_D_CUST_FIRST_NAME
,MAX(INDUSTRIAL_SECTOR) as INDUSTRIAL_SECTOR
,MAX(V_D_CUST_INDUSTRY) as V_D_CUST_INDUSTRY
,RPSIA_THIRD_PARTY
,SUM(OUTSTANDING) AS OUTSTANDING
,SUM(GROSS_LOAN) AS GROSS_LOAN_RPSIA
,SUM(OUTSTANDING_AFTER_EIR) AS OUTSTANDING_AFTER_EIR
,SUM(OUTSTANDING_AFTER_EIR_UWI) AS OUTSTANDING_AFTER_EIR_UWI

FROM RPSIA_ACCTLEVEL A
GROUP BY 1,2,3;""")

donn.commit()

IF_RPSIA_ACCTLEVEL2 = pd.read_sql_query("SELECT * FROM IF_RPSIA_ACCTLEVEL2", donn)

d.execute("""
CREATE TABLE IF_RPSIA_CUSLEVEL AS SELECT
CUSTOMER_ID
,V_D_CUST_FIRST_NAME
,MAX(INDUSTRIAL_SECTOR) as INDUSTRIAL_SECTOR
,MAX(V_D_CUST_INDUSTRY) as V_D_CUST_INDUSTRY
,SUM(OUTSTANDING) AS OUTSTANDING
,SUM(GROSS_LOAN) AS GROSS_LOAN_RPSIA
,SUM(OUTSTANDING_AFTER_EIR) AS OUTSTANDING_AFTER_EIR
,SUM(OUTSTANDING_AFTER_EIR_UWI) AS OUTSTANDING_AFTER_EIR_UWI
FROM RPSIA_ACCTLEVEL
GROUP BY 1,2;""")

donn.commit()

IF_RPSIA_CUSLEVEL = pd.read_sql_query("SELECT * FROM IF_RPSIA_CUSLEVEL", donn)

#======================================================EXPORT

writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\10.0 RPSIA\\'+str(date)[:4]+'\\RPSIA_'+str(date)[:6]+'(python).xlsx',engine='xlsxwriter')

RPSIA.to_excel(writer2, sheet_name='Export Worksheet', index = False)

RPSIA_ACCTLEVEL1.to_excel(writer2, sheet_name='LIST_RPSIA_ACCT', index = False)

IF_RPSIA_ACCTLEVEL2.to_excel(writer2, sheet_name='LIST_RPSIA_ACCT2', index = False)

IF_RPSIA_CUSLEVEL.to_excel(writer2, sheet_name='LIST_RPSIA_CUS', index = False)

writer2.save()

IF_RPSIA_ACCTLEVEL2.to_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\10.0 RPSIA\\'+str(date)[:4]+'\\IF_RPSIA_ACCTLEVEL2_'+str(date)+'.txt', index = False)



# DB SECTION
SQLite in Python (Outsource)

In [ ]:
d.execute("drop table IF_SUMM_NCIF_PDT")
donn.commit() 

In [34]:
donn.close()

In [ ]:
cek = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name ;", donn)
cek